In [31]:
import requests
import pandas as pd
import plotly.express as px

from enum import Enum
from decimal import Decimal
from dataclasses import dataclass

In [44]:
def get_bybit_spot():
    baseurl = "https://api.bybit.com"
    payload = { "category": "spot" }
    r = requests.get(baseurl + "/v5/market/tickers", params=payload)
    data = r.json()["result"]["list"]

    markets = []
    for pair in data:
        market = {"symbol" : pair["symbol"], "price": Decimal(pair["lastPrice"]), "volume": Decimal(pair["volume24h"])}
        markets.append(market)

    for market in markets:
        market["rvolume"] = market["price"] * market["volume"]

    markets.sort(key=lambda x: x.get("rvolume"), reverse=True)
    return markets
    

In [13]:
def get_bybit_futures():
    baseurl = "https://api.bybit.com"
    payload = { "category": "linear" }
    r = requests.get(baseurl + "/v5/market/tickers", params=payload)
    data = r.json()["result"]["list"]

    markets = []
    for pair in data:
        market = {"symbol" : pair["symbol"], "price": Decimal(pair["lastPrice"]), "oi": pair["openInterest"], "fr": pair["fundingRate"], "volume": Decimal(pair["volume24h"])}
        markets.append(market)

    for market in markets:
        market["rvolume"] = market["price"] * market["volume"]

    markets.sort(key=lambda x: x.get("rvolume"), reverse=True)
    return markets

In [39]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: int
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

def exluded_markets(check: str, excluded: list[str]) -> bool:
    if len(excluded) > 0:
        for item in excluded:
            if check.startswith(item) or check.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"

    def __init__(self) -> None:
        pass

    def markets(self, market: str, exclude: list[str] = []) -> list[str]:
        url = Binance.BASEURL + "/api/v3/ticker/price"
        r = requests.get(url)
        data = filter(lambda x: x.endswith(market) and not exluded_markets(x, exclude), map(lambda x: x.get("symbol", ""), r.json()))
        return list(data)

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + "/api/v3/klines"
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url)

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines


In [40]:
binance = Binance()
markets = binance.markets("USDT", ["TUSD", "BUSD", "USDC"])